In [12]:
### Activate virtual environment with bash

"""
cd flowers_classifier
python3 -m venv venv
. venv/bin/activate
pip install requirements.txt
"""

'\ncd flowers_classifier\npython3 -m venv venv\n. venv/bin/activate\npip install requirements.txt\n'

In [2]:
# Import modules

from collections import OrderedDict
import json

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.autograd import Variable

In [3]:
# Jupyter configuration

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
# Set names

data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [5]:
# Define transforms for training, validation, and testing datasets

data_transforms = {
    'train' : transforms.Compose([transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),transforms.RandomRotation(30),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                         [0.229, 0.224, 0.225])]),
                                                            
    'valid' : transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])]),

    'test' : transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])
}

In [6]:
# Load and transform datasets

image_datasets = {
    'train' : datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'test' : datasets.ImageFolder(test_dir, transform=data_transforms['test']),
    'valid' : datasets.ImageFolder(valid_dir, transform=data_transforms['valid'])
}

In [7]:
# Define data loaders

dataloaders = {
    'train' : torch.utils.data.DataLoader(image_datasets['train'], batch_size=64, shuffle=True),
    'test' : torch.utils.data.DataLoader(image_datasets['test'], batch_size=64, shuffle=False),
    'valid' : torch.utils.data.DataLoader(image_datasets['valid'], batch_size=64, shuffle=True)
}

In [8]:
# Define class to index mapping for training dataset

## key = directory
## value = index of directory

class_to_idx = image_datasets['train'].class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

In [9]:
# Load mapping for label to name from json

with open('class_to_name.json', 'r') as f:
    class_to_name = json.load(f)

In [10]:
# Test class_to_idx mapping

class_ = '34'  # should be pink primrose

print(f'''
Class Directory: {class_}
Class Index: {class_to_idx[class_]}
Class Name: {class_to_name[class_]}
''')


Class Directory: 34
Class Index: 30
Class Name: mexican aster



In [11]:
# Test mapping by image index or random image

from image_utils import inspect_image

image_metadata = inspect_image(
    dataset_name='train',
    dataloaders=dataloaders,
    class_to_idx=class_to_idx,
    class_to_name=class_to_name,
    # image_idx=255  # optional
    plot=False
)

print(image_metadata)

{'class': '93', 'class_index': 95, 'class_name': 'ball moss', 'image_path': 'flowers/train/93/image_06013.jpg', 'image_index': 6043}


In [13]:
# Load model from checkpoint

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = models.vgg13(pretrained = True)
    model.state_dict = checkpoint['state_dict']
    model.classifier = checkpoint['classifer']
    # model.classifier = checkpoint['classifier']    # it should be this fix me later
    model.class_to_idx = checkpoint['class_to_idx']
    
    return model

model = load_checkpoint('checkpoint.pth')
model

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.